In [2]:
# Bring in all of our dependencies
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import gym
import math



Using Theano backend.


In [9]:
# We should define the environment here and detremine the sizing so we can generalize our simple neural network
env    = gym.make('CartPole-v0')   # Choose our environment
tmp    = env.action_space.sample()
print(type(tmp))
action = np.array([tmp])
N      = len(action)               # Use this to generalize the input size for the NN
print(N)

[2016-12-02 17:07:41,621] Making new env: CartPole-v0


<class 'int'>
1


In [4]:
# Make a very simple neural network, reLU layers have been shown to train faster. We only have one hidden layer here, very simple
model = Sequential()
model.add(Dense(100, input_shape=((4,)), activation='relu',init='uniform'))
model.add(Dense(100, activation='relu',init='uniform'))
model.add(Dense(2))

In [5]:
# Build the model for training, we can tweak our loss function and optimizer for different environments
model.compile(loss='mean_squared_error', optimizer='adam')

In [14]:
# Initialize replay memory
#repMem = np.empty((0,4))
runAvg = np.zeros((5,1))
gamma = 0.7
epsUpdate = 0.05
maxMem = 1000
obs1 = env.reset()
action = env.action_space.sample()
obs2, reward, done, info = env.step(action)
repMem = np.array([obs1,action,reward,obs2,done])
tFlag = 0
fSkip = 0

#Start monigtoring
#env.monitor.start('/tmp/cartpole-experiment-1')

#repMem = [1,2,3,4]
# Run episodes tally results
for i_episode in range(30):
    obs1 = env.reset()
    epsilon = epsUpdate * i_episode
    rTally = np.array([0])
    loss = 0.
    
    for t in range(451):
        env.render()
        obs2 = obs1
        
        # Add action selection by max output of network over time, update epsilon slowly
        if np.random.uniform(0,1,1) >= min(0.9,epsilon):
            action = env.action_space.sample()                    # this is a random action
        else:
            action = int(np.argmax(model.predict(obs2[np.newaxis,])[0]))  
            
        obs1, reward, done, info = env.step(action)    # apply the action to the model and check the outputs
        
        # Quit early if too many steps
        if t == 450:
            done = True
        
        # Training Stuff ===========================================================================
        if tFlag == 0:
            repMem = np.column_stack((repMem, np.array([obs2,action,reward,obs1,done])))

            if repMem.shape[1] > maxMem:
                #np.delete(repMem,repMem.shape[1]-1,1)
                np.delete(repMem,0,1)

            if repMem.shape[1] > 10:
                P = max(int(1),math.floor(repMem.shape[1]/5))
            else:
                P = int(1)

            # Get random batch
            memBatch = repMem[:,np.random.choice(repMem.shape[1], int(P), replace=False)]


            # Put Q-Learning stuff here to start...
            # Q[s,a] = Q[s,a] + a(r + y max_a'(Q[s',a'] - Q[s,a]))
            # Get targets
            tgt = np.zeros((P,2))

            for batchNum in range(P):

                tmp0 = memBatch[0,batchNum]
                tmp0 = tmp0[np.newaxis,]
                tgt[batchNum,:] = model.predict(tmp0)[0]

                tmp = memBatch[3,batchNum]
                tmp = tmp[np.newaxis,]
                QVal = np.max(model.predict(tmp)[0])

                tmpReward = memBatch[2,batchNum]
                tmpA      = memBatch[1,batchNum]

                tmpDone = memBatch[4,batchNum]
                if tmpDone:
                    tgt[batchNum,tmpA] = tmpReward
                else:
                    tgt[batchNum,tmpA] = tmpReward + (gamma * QVal)     
                
            tFlag = 1
                
            # Train              
            if memBatch[0,:].shape[0] == 1:
                tmp = memBatch[0,]
                tmp = tmp[0]
                tmp = tmp[np.newaxis,]
            else:
                tmp = np.vstack(memBatch[0,:])

            loss += model.train_on_batch(tmp, tgt)
            
        else:
            tFlag = 0
          
        # ========================================================================

        if done:
            print("Episode finished after {} timesteps".format(t+1))

            if i_episode <= 3:
                runAvg[i_episode] = t+1
            else:
                runAvg[0:3] = runAvg[1:4]
                runAvg[4] = t+1
                #print("Running Average: {} ".format(np.sum(runAvg)/5)) 
                #if np.sum(runAvg)/5 >=195:
                    #tFlag = 1
        
            break
            
model.save('my_model.h5')            
#env.monitor.close()

Episode finished after 13 timesteps
Episode finished after 10 timesteps
Episode finished after 18 timesteps
Episode finished after 16 timesteps
Episode finished after 19 timesteps
Episode finished after 42 timesteps
Episode finished after 56 timesteps
Episode finished after 12 timesteps
Episode finished after 21 timesteps
Episode finished after 77 timesteps
Episode finished after 11 timesteps
Episode finished after 15 timesteps
Episode finished after 25 timesteps
Episode finished after 21 timesteps
Episode finished after 37 timesteps
Episode finished after 48 timesteps
Episode finished after 64 timesteps
Episode finished after 369 timesteps
Episode finished after 345 timesteps
Episode finished after 396 timesteps
Episode finished after 351 timesteps
Episode finished after 341 timesteps
Episode finished after 451 timesteps
Episode finished after 271 timesteps
Episode finished after 440 timesteps
Episode finished after 451 timesteps
Episode finished after 422 timesteps
Episode finished a